# Introduction to Semantic Kernel Agents

Welcome to this workshop on Semantic Kernel (SK) agents! In this notebook, we'll explore how to create, configure, and use agents with the Semantic Kernel framework.

## What You'll Learn
- What agents are in Semantic Kernel
- How to create and configure different types of agents
- Basic and advanced interaction patterns with agents
- How to integrate plugins and functions with agents
- Multi-agent systems and orchestration

Let's start by setting up our environment and understanding the foundational concepts of Semantic Kernel agents.

## 1. Introduction to Semantic Kernel Agents

### What are agents in Semantic Kernel?

In Semantic Kernel, an **agent** is a specialized component that can interact with Large Language Models (LLMs), process conversations, make decisions, and potentially execute code or call functions. Unlike simple prompt-based interactions, agents maintain state, follow instructions, and can engage in multi-turn conversations to accomplish tasks.

At their core, agents are designed to:
- Process user inputs and generate contextual responses
- Maintain conversation history and context
- Execute functions when appropriate (function calling)
- Work independently or collaborate with other agents


### Agent Architecture in Semantic Kernel

Key components in the agent architecture:

1. **Kernel**: The core orchestration engine that connects agents to AI services and functions
2. **AI Service**: The underlying LLM that powers the agent (like Azure OpenAI, OpenAI)
3. **Chat History**: Maintains the conversation context over multiple turns
4. **Plugins/Functions**: Optional extensions that allow the agent to perform specific tasks

### Types of Agents in Semantic Kernel

Semantic Kernel primarily offers two types of agents:

1. **ChatCompletionAgent**
   - Lightweight agent that uses your kernel's chat completion service
   - Good for simple conversational tasks
   - Manages chat history locally

2. **OpenAIAssistantAgent / AzureAssistantAgent**
   - Uses OpenAI's Assistant API
   - Maintains conversation state remotely as "threads"
   - Supports advanced features like code interpretation and file searching
   - Requires explicit thread management

Let's first install the necessary packages to work with Semantic Kernel agents:


In [ ]:
pip install -r requirements.txt

Now, let's set up our environment by loading environment variables and importing the necessary modules:

In [ ]:
import os
import asyncio
from dotenv import load_dotenv

# Import Semantic Kernel components
import semantic_kernel as sk
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    OpenAIChatCompletion,
)
from semantic_kernel.contents import ChatHistory, ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.functions import KernelArguments
from semantic_kernel.connectors.ai import FunctionChoiceBehavior
# Load environment variables from .env file
load_dotenv()

For this workshop, we'll need to set up our environment variables with either Azure OpenAI or OpenAI credentials. You can create a `.env` file in the same directory as this notebook with the following variables:

```
AZURE_OPENAI_ENDPOINT='[YOUR_ENDPOINT]'
AZURE_OPENAI_API_KEY='[YOUR_API_KEY]'
AZURE_OPENAI_MODEL_DEPLOYMENT_NAME='gpt-4o'

```

## 2. Setting Up Your Environment

Now that we understand what agents are and have imported the necessary modules, let's begin by creating our first kernel instance and configuring the environment properly.

### Creating a Kernel Instance

The `Kernel` is the central orchestration component in Semantic Kernel. It manages AI services, plugins, and other resources that agents need to function. Let's now create a helper function that will configure a kernel with the appropriate AI service based on the available credentials:

In [ ]:
service_id = 'agent'
def create_kernel_with_service(service_id):
    kernel = sk.Kernel()
    kernel.add_service(
        AzureChatCompletion(
            service_id=service_id,
            deployment_name=os.getenv("AZURE_OPENAI_MODEL_DEPLOYMENT_NAME"),
            api_key=os.getenv('AZURE_OPENAI_API_KEY'),
            endpoint=os.getenv('AZURE_OPENAI_ENDPOINT')
        )
    )
    return kernel

Now let's create our first kernel:

In [ ]:
kernel = create_kernel_with_service(service_id="chat-completion")
print("Kernel created successfully!")

### Understanding Service Configuration

When adding an AI service to the kernel, we specified a `service_id`. This ID is important because:

1. It allows you to add multiple services to the same kernel
2. You can selectively use different services for different agents
3. It helps organize and identify your services

If you need specific execution settings for your AI service (like temperature, top-p, or function calling behavior), you can retrieve and modify them:

In [ ]:
# Get the execution settings for our service
settings = kernel.get_prompt_execution_settings_from_service_id(
    service_id="chat-completion"
)

# Configure settings
settings.temperature = 0.7
settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

### Exercise: Create Multiple Services in a Kernel

In this exercise, try creating a kernel with two different AI services with different configurations. This will be useful when we want to use different services for different agents or functions.

Your task:
1. Create a new kernel
2. Add two services with different service IDs (e.g., "creative" and "precise")
3. Configure the "creative" service with higher temperature (e.g., 0.8)
4. Configure the "precise" service with lower temperature (e.g., 0.2)

In [ ]:
# Initialize your kernel

# Add first service - creative with higher temperature

# Add second service - precise with lower temperature

# Configure the settings for each service


<details>
    <summary>Click to see solution</summary>

    ```python
    # Create a new kernel for multiple services
    multi_service_kernel = sk.Kernel()

    # Add first service - creative with higher temperature

    multi_service_kernel.add_service(
        AzureChatCompletion(
            service_id="creative",
            deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
            endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_key=os.getenv("AZURE_OPENAI_API_KEY")
        )
    )

    # Add second service - precise with lower temperature
    multi_service_kernel.add_service(
        AzureChatCompletion(
            service_id="precise",
            deployment_name=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
            endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_key=os.getenv("AZURE_OPENAI_API_KEY")
        )
    )

    # Configure the settings for each service
    creative_settings = multi_service_kernel.get_prompt_execution_settings_from_service_id(service_id="creative")
    creative_settings.temperature = 0.8
    print(f"Creative service temperature: {creative_settings.temperature}")

    precise_settings = multi_service_kernel.get_prompt_execution_settings_from_service_id(service_id="precise")
    precise_settings.temperature = 0.2
    print(f"Precise service temperature: {precise_settings.temperature}")
    ```
</details>

### Key Takeaways

In this section, we've learned how to:

1. **Set up our environment** with necessary imports and credentials
2. **Create a kernel** as the foundation for our agents
3. **Configure AI services** with specific IDs and settings
4. **Manage execution settings** to control how the AI generates responses

In the next section, we'll create our first agent using the kernel we've just configured.

## 3. Your First Agent: ChatCompletionAgent

Now that we have our kernel and services set up, let's create our first agent. We'll start with the `ChatCompletionAgent`, which is a simple yet powerful agent that leverages the chat completion capabilities of large language models.

### Creating a Basic Agent

To create a `ChatCompletionAgent`, we need to provide:
1. A kernel with a configured chat service
2. Instructions that define the agent's behavior
3. Optional parameters like a name and execution settings

Let's create a simple assistant agent:

In [ ]:
# Create a simple assistant agent
assistant_agent = ChatCompletionAgent(
    kernel=kernel,
    name="Assistant",
    instructions="You are a helpful assistant that provides concise and accurate information. Keep your responses brief but informative.",
)

print(f"Agent '{assistant_agent.name}' created successfully!")

### Configuring Instructions and Parameters

The `instructions` parameter is crucial as it defines how your agent will behave. Think of it as the "system prompt" that shapes the agent's personality, capabilities, and limitations. Let's explore some more complex instructions:

In [ ]:
math_tutor_agent = ChatCompletionAgent(
    kernel=kernel,
    name="MathTutor",
    instructions="""You are a math tutor specialized in helping students understand mathematical concepts.
    
    When responding to questions:
    1. First explain the underlying concept in simple terms
    2. Then walk through the solution step by step
    3. Provide a simple example to reinforce the learning
    4. Avoid solving problems directly without explanation
    
    Always be encouraging and patient.
    """,
)

print(f"Agent '{math_tutor_agent.name}' created with specialized instructions.")

### Understanding Agent Execution

Once an agent is created, it needs a chat history to interact with. The chat history maintains the state of the conversation and provides context for the agent's responses.

Here's a simple example of how to execute an agent:

In [ ]:
# Create a chat history to maintain conversation state
chat_history = ChatHistory()

# Add a user message to the chat history
chat_history.add_message(
    ChatMessageContent(
        role=AuthorRole.USER, content="Hello! Can you introduce yourself?"
    )
)


# Define a function to execute the agent asynchronously
async def get_agent_response(agent, history):
    # Get a single response from the agent
    response = await agent.get_response(messages=history)
    return response


# Execute the agent
response = await get_agent_response(assistant_agent, chat_history)

# Print the agent's response
print(f"Agent: {response.content}")

There are actually three main ways to invoke an agent:

1. **`get_response()`**: Returns a single response directly as a `ChatMessageContent` object
2. **`invoke()`**: Returns an async iterable of `ChatMessageContent` objects
3. **`invoke_stream()`**: Streams the response in real-time (useful for long responses)

Let's see how `invoke()` works:

In [ ]:
# Create a new chat history
chat_history = ChatHistory()
chat_history.add_message(
    ChatMessageContent(
        role=AuthorRole.USER, content="What can you tell me about quantum computing?"
    )
)


# Define a function to invoke the agent using invoke()
async def invoke_agent(agent, history):
    responses = []
    # Iterate through the responses asynchronously
    async for response in agent.invoke(messages=history):
        responses.append(response)
    return responses


# Execute the agent
responses = await invoke_agent(assistant_agent, chat_history)

# Print the responses
for i, response in enumerate(responses):
    # Access the message content correctly through the AgentResponseItem
    message_text = response.message.content
    print(f"Response {i + 1}: {message_text}")

### Exercise: Implement a Simple Question-Answering Agent

Now it's your turn to create a custom agent. Implement a question-answering agent that specializes in providing factual information about a specific topic of your choice.

Your task:
1. Create a new `ChatCompletionAgent` with a descriptive name
2. Configure it with detailed instructions that define its area of expertise and how it should respond
3. Create a chat history with a relevant question
4. Execute the agent and display its response

In [ ]:
# Your code here to create and execute a question-answering agent

# 1. Create your agent with specialized instructions

# 2. Create a chat history with a relevant question

# 3. Execute the agent and get its response

# 4. Print the response

<details>
<summary>Click to see solution</summary>

```python
# Create a specialized space expert agent with detailed instructions
space_expert_agent = ChatCompletionAgent(
    kernel=kernel,
    name="SpaceExpert",
    instructions="""You are an expert in astronomy and space exploration.
    
    When answering questions:
    - Provide factual, scientifically accurate information
    - Include relevant dates, measurements, and statistics when applicable
    - Explain complex concepts in accessible language
    - Differentiate between established facts and theoretical or speculative ideas
    - When appropriate, mention recent developments or missions
    
    Focus on being educational and inspiring curiosity about space.
    """
)

# Create a chat history with a specific astronomy question
space_chat = ChatHistory()
space_chat.add_message(ChatMessageContent(
    role=AuthorRole.USER, 
    content="What are exoplanets and how do scientists detect them?"
))

# Helper function to get the agent's response
async def get_expert_response(agent, history):
    response = await agent.get_response(messages=history)
    return response

# Execute the agent and get its response
space_response = await get_expert_response(agent=space_expert_agent, history=space_chat)

# Display the agent's response
print(f"SpaceExpert: {space_response.content}")
```
</details>